In [78]:
import requests, xmltodict, pandas as pd
from pandas import json_normalize
from zeep import Client, helpers
from datetime import datetime
import ast
import os
from pathlib import Path
import re
from typing import Optional
import re
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
from typing import Dict, List

In [3]:
def safe_serialize(obj):
    # tipos base
    if obj is None or isinstance(obj, (str, int, float, bool)):
        return obj
    if isinstance(obj, list):
        return [safe_serialize(x) for x in obj]
    if isinstance(obj, dict):
        return {k: safe_serialize(v) for k, v in obj.items()}

    # intentar con zeep.helpers.serialize_object (si está disponible)
    try:
        from zeep.helpers import serialize_object
        ser = serialize_object(obj)
        if ser is not None:
            return safe_serialize(ser)  # recursivo por si trae más objetos
    except Exception:
        pass

    # objetos Zeep suelen tener __values__
    vals = getattr(obj, "__values__", None)
    if isinstance(vals, dict):
        return {k: safe_serialize(v) for k, v in vals.items()}

    # fallback genérico
    try:
        return {k: safe_serialize(v) for k, v in vars(obj).items()}
    except Exception:
        return obj

def listar_ops(wsdl_url):
    c = Client(wsdl=wsdl_url)
    for svc in c.wsdl.services.values():
        for port in svc.ports.values():
            binding = port.binding
            for name, op in binding._operations.items():
                # Nombre de la operación
                args = []
                if op.input and op.input.body and op.input.body.type:
                    args = [elt[0] for elt in op.input.body.type.elements]  # nombres de parámetros
                print(f"{name}({', '.join(args)})")

In [3]:
listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")

retornarPeriodosLegislativos()
retornarPeriodoLegislativoActual()
retornarLegislaturas()
retornarLegislaturaActual()
retornarMensajesXAnno(prmAnno)
retornarMocionesXAnno(prmAnno)
retornarMaterias()
retornarTramitesConstitucionales()
retornarTramitesReglamentarios()
retornarVotacionesXProyectoLey(prmNumeroBoletin)
retornarVotacionesXAnno(prmAnno)
retornarVotacionDetalle(prmVotacionId)
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)
retornarPeriodosLegislativos()
retornarPeriodoLegislativoActual()
retornarLegislaturas()
retornarLegislaturaActual()
retornarMensajesXAnno(prmAnno)
retornarMocionesXAnno(prmAnno)
retornarMaterias()
retornarTramitesConstitucionales()
retornarTramitesReglamentarios()
retornarVotacionesXProyectoLey(prmNumeroBoletin)
retornarVotacionesXAnno(prmAnno)
retornarVotacionDetalle(prmVotacionId)
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)
retornarPeriodosLegislativos()
retornarPeriodoLegislativoAct

In [4]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarPeriodosLegislativos()
print(res)

[{
    'Id': 1,
    'Nombre': '1990-1994',
    'FechaInicio': datetime.datetime(1990, 3, 11, 0, 0),
    'FechaTermino': datetime.datetime(1994, 3, 10, 0, 0),
    'Legislaturas': {
        'Legislatura': [
            {
                'Id': 3,
                'Numero': 319,
                'FechaInicio': datetime.datetime(1990, 3, 11, 0, 0),
                'FechaTermino': datetime.datetime(1990, 5, 20, 23, 59, 59),
                'Tipo': {
                    '_value_1': 'Extraordinaria',
                    'Valor': 2
                }
            },
            {
                'Id': 4,
                'Numero': 320,
                'FechaInicio': datetime.datetime(1990, 5, 21, 0, 0),
                'FechaTermino': datetime.datetime(1990, 9, 17, 23, 59, 59),
                'Tipo': {
                    '_value_1': 'Ordinaria',
                    'Valor': 1
                }
            },
            {
                'Id': 5,
                'Numero': 321,
                'Fec

In [7]:
d = safe_serialize(res) or {}
df = pd.json_normalize(d, sep=".")
df

,Id,Nombre,FechaInicio,FechaTermino,Diputados,Legislaturas.Legislatura,Legislaturas
0,1,1990-1994,1990-03-11,1994-03-10 00:00:00,None,"[{'Id': 3, 'Numero': 319, 'FechaInicio': 1990-...",NaN
1,2,1994-1998,1994-03-11,1998-03-10 00:00:00,None,"[{'Id': 12, 'Numero': 328, 'FechaInicio': 1994...",NaN
2,3,1998-2002,1998-03-11,2002-03-10 00:00:00,None,"[{'Id': 21, 'Numero': 337, 'FechaInicio': 1998...",NaN
3,4,2002-2006,2002-03-11,2006-03-10 00:00:00,None,"[{'Id': 30, 'Numero': 346, 'FechaInicio': 2002...",NaN
4,5,2006-2010,2006-03-11,2010-03-10 00:00:00,None,"[{'Id': 40, 'Numero': 356, 'FechaInicio': 2008...",NaN
5,6,2010-2014,2010-03-11,2014-03-10 00:00:00,None,"[{'Id': 42, 'Numero': 358, 'FechaInicio': 2010...",NaN
6,7,1965-1969,1965-03-11,1969-03-11 00:00:00,None,NaN,NaN
7,8,2014-2018,2014-03-11,2018-03-10 23:59:59,None,"[{'Id': 48, 'Numero': 364, 'FechaInicio': 2016...",NaN
8,9,2018-2022,2018-03-11,2022-03-10 23:59:59,None,"[{'Id': 50, 'Numero': 366, 'FechaInicio': 2018...",NaN
9,10,2022-2026,2022-03-11,2026-03-10 23:59:59,None,"[{'Id': 54, 'Numero': 370, 'FechaInicio': 2022...",NaN


### Save legislatures

In [8]:
df_explode = df.explode("Legislaturas.Legislatura").reset_index(drop=True)
df_legislaturas = json_normalize(df_explode["Legislaturas.Legislatura"])
df_legislaturas.head(5)

,Id,Numero,FechaInicio,FechaTermino,Tipo._value_1,Tipo.Valor
0,3.0,319.0,1990-03-11,1990-05-20 23:59:59,Extraordinaria,2.0
1,4.0,320.0,1990-05-21,1990-09-17 23:59:59,Ordinaria,1.0
2,5.0,321.0,1990-09-18,1991-05-20 23:59:59,Extraordinaria,2.0
3,6.0,322.0,1991-05-21,1991-09-17 23:59:59,Ordinaria,1.0
4,7.0,323.0,1991-09-18,1992-05-20 23:59:59,Extraordinaria,2.0


In [9]:
df_legislaturas.rename(columns={
    "Id" : "Legislatura.Id",
    "FechaInicio" : "Legislatura.FechaInicio",
    "FechaTermino" : "Legislatura.FechaTermino"
}, inplace=True)

df_legislaturas.head(5)

,Legislatura.Id,Numero,Legislatura.FechaInicio,Legislatura.FechaTermino,Tipo._value_1,Tipo.Valor
0,3.0,319.0,1990-03-11,1990-05-20 23:59:59,Extraordinaria,2.0
1,4.0,320.0,1990-05-21,1990-09-17 23:59:59,Ordinaria,1.0
2,5.0,321.0,1990-09-18,1991-05-20 23:59:59,Extraordinaria,2.0
3,6.0,322.0,1991-05-21,1991-09-17 23:59:59,Ordinaria,1.0
4,7.0,323.0,1991-09-18,1992-05-20 23:59:59,Extraordinaria,2.0


In [10]:
df_legislaturas = pd.concat([df_explode, df_legislaturas], axis=1)
df_legislaturas.drop(["Diputados", "Legislaturas.Legislatura", "Legislaturas", 
                      "FechaInicio", "FechaTermino", "Tipo.Valor"], axis=1, inplace=True)
df_legislaturas.rename(columns={"Nombre" : "Periodo"}, inplace=True)
df_legislaturas.head()

,Id,Periodo,Legislatura.Id,Numero,Legislatura.FechaInicio,Legislatura.FechaTermino,Tipo._value_1
0,1,1990-1994,3.0,319.0,1990-03-11,1990-05-20 23:59:59,Extraordinaria
1,1,1990-1994,4.0,320.0,1990-05-21,1990-09-17 23:59:59,Ordinaria
2,1,1990-1994,5.0,321.0,1990-09-18,1991-05-20 23:59:59,Extraordinaria
3,1,1990-1994,6.0,322.0,1991-05-21,1991-09-17 23:59:59,Ordinaria
4,1,1990-1994,7.0,323.0,1991-09-18,1992-05-20 23:59:59,Extraordinaria


In [6]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

def sanitize(name):
    """Limpia nombres de carpeta/archivo."""
    if pd.isna(name) or str(name).strip() == "":
        return "sin_periodo"
    return re.sub(r"[^\w\-]+", "_", str(name)).strip("_")

for col in ["Legislatura.FechaInicio", "Legislatura.FechaTermino"]:
    if col in df_legislaturas.columns:
        df_legislaturas[col] = pd.to_datetime(df_legislaturas[col], errors="coerce")

cols = [c for c in [
    "Legislatura.Id", "Numero",
    "Legislatura.FechaInicio", "Legislatura.FechaTermino",
    "Tipo._value_1", "Tipo.Valor"
] if c in df_legislaturas.columns]

# === 1) Split por periodo y guardar ===
for periodo, sub in df_legislaturas.groupby("Periodo", dropna=False):
    carpeta = DATA_DIR / sanitize(periodo)      # p.ej. data/1990-1994
    carpeta.mkdir(parents=True, exist_ok=True)

    # ordenar por fecha y número
    order_cols = [c for c in ["Legislatura.FechaInicio", "Numero"] if c in sub.columns]
    sub_sorted = sub.sort_values(order_cols) if order_cols else sub

    # seleccionar columnas, si no, guarda todo
    to_save = sub_sorted[cols] if cols else sub_sorted

    # guardar
    out_path = carpeta / "legislaturas.csv"
    to_save.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✓ Guardado {out_path}")

NameError: name 'df_legislaturas' is not defined

In [11]:
df_periodos = (
    df_legislaturas[["Id", "Periodo"]]
    .drop_duplicates()
    .set_index("Id")
)
df_periodos

,Periodo
Id,
1,1990-1994
2,1994-1998
3,1998-2002
4,2002-2006
5,2006-2010
6,2010-2014
7,1965-1969
8,2014-2018
9,2018-2022


### Save parliamentarians

In [64]:
def estandarizar_df_diputados_x_periodo(client: Client, period_id: int, df_periodos: pd.DataFrame) -> pd.DataFrame:
    """
      - Mantiene sub-tramos de militancia dentro del período legislativo (cambios de partido).
      - Asigna periodo_id y Periodo por TRASLAPE con el período oficial (df_periodos).
      - Calcula militancias_anteriores (inicio < inicio_oficial_período).
      - Calcula edad al inicio oficial del período (o fallback 11-03).
      - Drop/rename y orden final de columnas para CSV.
    """

    # ===== 0) Normaliza df_periodos y saca la fila del período solicitado =====
    dfp = df_periodos.reset_index().copy()
    # Detecta columnas
    # ID
    cand_id = [c for c in dfp.columns if c in ["Id", "ID", "id", "periodo_id", "Id_periodo"]]
    if not cand_id:
        raise ValueError(f"df_periodos no tiene columna ID reconocible. Columnas: {list(dfp.columns)}")
    id_col = cand_id[0]
    # Label
    cand_label = [c for c in dfp.columns if c in ["Periodo", "Nombre", "NombrePeriodo", "periodo", "nombre"]]
    if not cand_label:
        raise ValueError(f"df_periodos no tiene columna de nombre/etiqueta de período. Columnas: {list(dfp.columns)}")
    label_col = cand_label[0]
    # Fechas (pueden o no existir)
    # Si faltan, derivamos (YYYY-YYYY) -> [YYYY-03-11, YYYY2-03-10]
    if "FechaInicio" in dfp.columns:
        dfp["FechaInicio"] = pd.to_datetime(dfp["FechaInicio"], errors="coerce")
    else:
        dfp["FechaInicio"] = pd.NaT
    if "FechaTermino" in dfp.columns:
        dfp["FechaTermino"] = pd.to_datetime(dfp["FechaTermino"], errors="coerce")
    else:
        dfp["FechaTermino"] = pd.NaT

    def _derive_dates_from_label(lbl):
        try:
            a, b = str(lbl).split("-")
            y1, y2 = int(a), int(b)
            ini = pd.Timestamp(f"{y1}-03-11")
            fin = pd.Timestamp(f"{y2}-03-10")
            return ini, fin
        except Exception:
            return pd.NaT, pd.NaT

    # Completa fechas faltantes desde el label
    mask_need_ini = dfp["FechaInicio"].isna()
    mask_need_fin = dfp["FechaTermino"].isna()
    if mask_need_ini.any() or mask_need_fin.any():
        der = dfp[label_col].apply(_derive_dates_from_label)
        dfp.loc[mask_need_ini, "FechaInicio"] = [t[0] for t in der[mask_need_ini]]
        dfp.loc[mask_need_fin, "FechaTermino"] = [t[1] for t in der[mask_need_fin]]

    # Saca período oficial solicitado
    rowp = dfp.loc[dfp[id_col] == period_id]
    if rowp.empty:
        raise ValueError(f"No encontré el período con Id={period_id} en df_periodos.")
    periodo_label_oficial = str(rowp.iloc[0][label_col])
    inicio_oficial = rowp.iloc[0]["FechaInicio"]
    fin_oficial = rowp.iloc[0]["FechaTermino"]

    # ===== 1) fetch + normalize del SOAP =====
    res = client.service.retornarDiputadosXPeriodo(period_id)
    d = safe_serialize(res) or {}
    df = pd.json_normalize(d, sep=".")

    # ===== 2) Explota militancias =====
    df_explode = df.explode("Diputado.Militancias.Militancia").reset_index(drop=True)

    # ===== 3) Normaliza objeto militancia =====
    df_militancias = (
        json_normalize(df_explode["Diputado.Militancias.Militancia"])
        .rename(columns={
            "FechaInicio": "Militancia.FechaInicio",
            "FechaTermino": "Militancia.FechaTermino",
        })
    )
    
    # ===== 4) Concat + fechas a datetime =====
    df_dip = pd.concat([df_explode, df_militancias], axis=1)
    for col in ["Militancia.FechaInicio", "Militancia.FechaTermino"]:
        if col in df_dip.columns:
            df_dip[col] = pd.to_datetime(df_dip[col], errors="coerce")

    # ===== 5) Mantener SOLO militancias que SOLAPAN con el período oficial =====
    SENTINEL_FUTURO = pd.Timestamp("2099-12-31")
    ini_mil = df_dip["Militancia.FechaInicio"]
    fin_mil = df_dip["Militancia.FechaTermino"].fillna(SENTINEL_FUTURO)
    solapa = (ini_mil <= fin_oficial) & (fin_mil >= inicio_oficial)
    df_dip = df_dip.loc[solapa].copy().set_index("Diputado.Id")

    # --- helper para nombre completo ---
    def _nombre_completo(row):
        partes = [
            row.get("Diputado.Nombre"),
            row.get("Diputado.Nombre2"),
            row.get("Diputado.ApellidoPaterno"),
            row.get("Diputado.ApellidoMaterno"),
        ]
        return " ".join([p for p in partes if p and str(p).strip() != ""]).strip()
    
    # ===== 6) Calcula nombre completo y asigna periodo oficial =====    
    df_dip["nombre_completo"] = df_dip.apply(_nombre_completo, axis=1)
    df_dip["Id_periodo"] = period_id
    df_dip["Periodo"] = periodo_label_oficial

    # ===== 7) militancias_anteriores: inicio < inicio_oficial (cuenta por diputado) =====
    # Ojo: contamos sobre TODO el set (no sólo los que solapan), por eso usamos el df original exploteado
    # pero en caso de querer sólo este subset, podrías usar df_dip en lugar de df_explode+militancias.
    # Aquí lo haremos con todas las militancias disponibles en la respuesta:
    df_all = pd.concat([df_explode, df_militancias], axis=1).set_index("Diputado.Id")
    if "Militancia.FechaInicio" in df_all.columns:
        df_all["Militancia.FechaInicio"] = pd.to_datetime(df_all["Militancia.FechaInicio"], errors="coerce")
    prev_counts = (
        df_all.loc[df_all["Militancia.FechaInicio"] < inicio_oficial]
              .groupby(level=0)
              .size()
              .rename("total_militancias_anteriores")
              .astype(int)
    )
    df_dip["total_militancias_anteriores"] = df_dip.index.map(prev_counts).fillna(0).astype(int)

    # ===== 8) EDAD al inicio oficial del período =====
    # Si no hay fecha oficial (NaT), fallback 11-03 del año inicial del label
    def _ref_date(label, fallback_ini):
        if pd.notna(inicio_oficial):
            return inicio_oficial
        try:
            y_ini = int(str(label).split("-")[0])
            return pd.Timestamp(f"{y_ini}-03-11")
        except Exception:
            return pd.NaT

    df_dip["Diputado.FechaNacimiento"] = pd.to_datetime(df_dip.get("Diputado.FechaNacimiento"), errors="coerce")
    ref_date = _ref_date(periodo_label_oficial, inicio_oficial)
    def _edad_en(dob, ref):
        if pd.isna(dob) or pd.isna(ref):
            return pd.NA
        return int(ref.year - dob.year - ((ref.month, ref.day) < (dob.month, dob.day)))
    df_dip["edad"] = df_dip["Diputado.FechaNacimiento"].apply(lambda dob: _edad_en(dob, ref_date))

    # ===== 9) DROP columnas que ya no interesan =====
    cols_drop = [
        "Distrito.Comunas.Comuna",
        "Diputado.Militancias.Militancia",
        "Diputado.FechaDefucion",
        "Diputado.RUT",
        "Diputado.RUTDV",
        "Partido.Alias",
        "FechaInicio",                 # del tramo 'res' si existiera
        "FechaTermino",                # del tramo 'res' si existiera
        "Diputado.Nombre",
        "Diputado.Nombre2",
        "Diputado.ApellidoPaterno",
        "Diputado.ApellidoMaterno",
        "Diputado.Sexo.Valor"
    ]
    df_dip.drop(columns=[c for c in cols_drop if c in df_dip.columns], errors="ignore", inplace=True)

    # ===== 10) RENOMBRES =====
    rename_map = {
        "Id_periodo": "periodo_id",
        "Diputado.FechaNacimiento": "fecha_nacimiento",
        "Diputado.Sexo._value_1": "sexo",
        "Distrito.Numero": "distrito_numero",
        "Partido.Id": "partido_id",
        "Partido.Nombre": "partido_nombre",
        "total_militancias_anteriores": "militancias_anteriores",
    }
    df_dip = df_dip.rename(columns=rename_map)

    # Asegura 'diputado_id' como columna
    df_dip["diputado_id"] = df_dip.index

    # ===== 11) Tipos útiles =====
    if "fecha_nacimiento" in df_dip.columns:
        df_dip["fecha_nacimiento"] = pd.to_datetime(df_dip["fecha_nacimiento"], errors="coerce")
    for int_col in ["periodo_id", "diputado_id", "distrito_numero", "militancias_anteriores"]:
        if int_col in df_dip.columns:
            if df_dip[int_col].isna().any():
                df_dip[int_col] = pd.to_numeric(df_dip[int_col], errors="coerce").astype("Int64")
            else:
                df_dip[int_col] = pd.to_numeric(df_dip[int_col], errors="coerce").astype(int)

    # ===== 12) Orden final recomendado =====
    ordered_cols = [
        "periodo_id",
        "Periodo",            # label oficial (para carpeta)
        "diputado_id",
        "nombre_completo",
        "fecha_nacimiento",
        "sexo",
        "edad",
        "distrito_numero",
        "partido_id",
        "partido_nombre",
        "militancias_anteriores",
        "Militancia.FechaInicio",
        "Militancia.FechaTermino"
    ]
    for c in ordered_cols:
        if c not in df_dip.columns:
            df_dip[c] = pd.NA

    df_final = df_dip[ordered_cols].reset_index(drop=True)
    return df_final


In [75]:
def sanitize(name):
    """Limpia nombres de carpeta/archivo."""
    if pd.isna(name) or str(name).strip() == "":
        return "sin_periodo"
    return re.sub(r"[^\w\-]+", "_", str(name)).strip("_")

# --- Aseguramos que df_periodos tenga columnas 'Id' y 'Periodo' ---
df_periodos_sorted = df_periodos.reset_index().sort_values("Id")

c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx?WSDL")

for _, row in df_periodos_sorted.iterrows():
    pid = int(row["Id"])                # ID del período legislativo
    nombre_periodo = str(row["Periodo"])  # Nombre o etiqueta del período, usado para carpeta

    try:
        df_pid = estandarizar_df_diputados_x_periodo(c, pid, df_periodos)

        carpeta = DATA_DIR / sanitize(nombre_periodo)   # p.ej., data/1990-1994
        carpeta.mkdir(parents=True, exist_ok=True)

        # (opcional) ordenar antes de guardar
        order_cols = [c for c in ["nombre_completo", "partido_nombre", "diputado_id"] if c in df_pid.columns]
        df_pid_sorted = df_pid.sort_values(order_cols) if order_cols else df_pid

        out_path = carpeta / "diputados.csv"
        df_pid_sorted.to_csv(out_path, index=False, encoding="utf-8")
        print(f"✓ Guardado {out_path}")

    except Exception as e:
        print(f"✗ Período {pid} ({nombre_periodo}): {e}")

✗ Período 1 (1990-1994): El servidor no puede procesar la solicitud. ---> La conversión del tipo 'DBNull' en el tipo 'Integer' no es válida.
✗ Período 2 (1994-1998): El servidor no puede procesar la solicitud. ---> La conversión del tipo 'DBNull' en el tipo 'Integer' no es válida.
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\1998-2002\diputados.csv
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\2002-2006\diputados.csv
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\2006-2010\diputados.csv
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\2010-2014\diputados.csv
✗ 

In [76]:
df = pd.read_csv("data\\2018-2022\\diputados.csv")

,periodo_id,Periodo,diputado_id,nombre_completo,fecha_nacimiento,sexo,edad,distrito_numero,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino
0,9,2018-2022,897,Alejandra Sepúlveda Orbenes,1965-11-13,Femenino,52,NaN,FRVS,Federación Regionalista Verde Social,4,2018-03-11 00:00:00,2022-03-10 23:59:59
1,9,2018-2022,1013,Alejandro Bernales Maldonado,1979-02-02,Masculino,39,NaN,LIBERAL,Partido Liberal de Chile,0,2018-03-11 00:00:00,2022-03-10 23:59:59
2,9,2018-2022,952,Alejandro Santana Tirachini,1965-07-18,Masculino,52,NaN,RN,Renovación Nacional,2,2018-03-11 00:00:00,2022-03-10 23:59:59
3,9,2018-2022,1076,Alexis Sepúlveda Soto,1969-12-29,Masculino,48,NaN,PR,Partido Radical de Chile,0,2018-03-11 00:00:00,2022-03-10 23:59:59
4,9,2018-2022,1043,Amaro Labra Sepúlveda,1949-10-25,Masculino,68,NaN,PC,Partido Comunista,0,2018-03-11 00:00:00,2022-03-10 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,9,2018-2022,991,Vlado Mirosevic Verdugo,1987-05-23,Masculino,30,NaN,LIBERAL,Partido Liberal de Chile,1,2018-03-11 00:00:00,2022-03-10 23:59:59
186,9,2018-2022,957,Víctor Torres Jeldes,1975-10-11,Masculino,42,NaN,DC,Partido Demócrata Cristiano,2,2018-03-11 00:00:00,2022-03-10 23:59:59
187,9,2018-2022,1059,Ximena Ossandón Irarrázabal,1963-12-13,Femenino,54,NaN,RN,Renovación Nacional,0,2018-03-11 00:00:00,2022-03-10 23:59:59
188,9,2018-2022,1017,Álvaro Carter Fernández,1980-10-09,Masculino,37,NaN,IND,Independientes,0,2020-09-29 00:00:00,2022-03-10 23:59:59


In [108]:
# --- 1) Helpers de normalización ---
def norm_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def pick_field(txt: str, label_regex: str):
    """
    Devuelve el valor que sigue a una etiqueta (ej: 'Partido', 'Región', etc.),
    cortando antes de barras verticales, bullets, saltos, o el inicio de otro campo.
    """
    # Permite :, -, – como separadores tras la etiqueta
    m = re.search(label_regex + r"\s*[:\-–]\s*([^|•\n\r]+)", txt, flags=re.I)
    if not m:
        # fallback si no hay separador claro: tomar lo que sigue hasta el próximo campo conocido
        m = re.search(label_regex + r"\s*(.*?)(?:\s+(?:Comit[eé]|Partido|Regi[oó]n|Circunscripci[oó]n)\b|$)", txt, flags=re.I)
    return norm_spaces(m.group(1)) if m else None

def parse_card_text(txt: str) -> dict:
    """
    txt de ejemplo:
    'Carmen Gloria Aravena Acuña Circunscripción 11 Región de La Araucanía Partido Independiente Comité Partido Renovación Nacional e Independientes'
    """
    t = norm_spaces(txt)

    # Nombre completo: todo lo que va antes de la primera etiqueta conocida
    # (Circunscripción|Región|Partido|Comité)
    m = re.match(r"^(.*?)\s+(?=(?:Circunscripci[oó]n|Regi[oó]n|Partido|Comit[eé])\b)", t, flags=re.I)
    nombre = norm_spaces(m.group(1)) if m else None

    circ = pick_field(t, r"Circunscripci[oó]n")
    region = pick_field(t, r"Regi[oó]n")
    partido = pick_field(t, r"Partido")
    comite  = pick_field(t, r"Comit[eé]")

    # Si 'Partido' aparece dos veces (p.ej. 'Partido Independiente' y luego 'Comité Partido ...'),
    # nos quedamos con la primera ocurrencia como partido real.
    # (El pick_field ya corta en la siguiente etiqueta, así que normalmente queda bien.)

    # Circunscripción: intenta extraer número si te sirve (pero guardamos el texto completo también)
    circ_num = None
    if circ:
        mnum = re.search(r"(\d+)", circ)
        circ_num = int(mnum.group(1)) if mnum else None

    return {
        "nombre_completo": nombre,
        "circunscripcion": circ,
        "circunscripcion_numero": circ_num,
        "region": region,
        "partido": partido,
        "comite": comite,
    }

# --- 2) Uso sobre cada div tarjeta ---
# Supongamos que ya tienes 'cards' con: soup.select("div.col-md-6.col-lg-3.col-12.mb-4")
def parse_cards(cards) -> pd.DataFrame:
    filas = []
    for card in cards:
        # quita posible basura y extrae texto "compacto"
        for bad in card(["script", "style"]): bad.extract()
        texto = card.get_text(" ", strip=True)
        filas.append(parse_card_text(texto))
    return pd.DataFrame(filas)

In [109]:
html = requests.get(URL, headers={"User-Agent":"Mozilla/5.0"}).text
soup = BeautifulSoup(html, "html.parser")

cards = soup.select("div.col-md-6.col-lg-3.col-12.mb-4")


df = parse_cards(cards)
df.head()

,nombre_completo,circunscripcion,circunscripcion_numero,region,partido,comite
0,Carmen Gloria Aravena Acuña,11,11,de La Araucanía,Independiente,Partido Renovación Nacional e Independientes
1,Pedro Araya Guerrero,3,3,de Antofagasta,P.P.D.,Partido Por la Democracia e Independientes
2,Karim Bianchi Retamales,15,15,de Magallanes y la Antártica Chilena,Independiente,Independiente I
3,Fabiola Campillai Rojas,7,7,Metropolitana de Santiago,Independiente,Mixto
4,Loreto Carvajal Ambiado,16,16,de Ñuble,P.P.D.,Partido Por la Democracia e Independientes


### Detalles de votación

In [15]:
def get_detalle(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionDetalle(idx)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df
    
def get_vot_year(year):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionesXAnno(year)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df

In [16]:
from __future__ import annotations

from pathlib import Path
import re
import pandas as pd
from pandas import json_normalize  # o usa pd.json_normalize si prefieres
from typing import Iterable, Optional

# === Config ===
PERIODO_RE = re.compile(r"^\s*(\d{4})\s*-\s*(\d{4})\s*$")

def parse_periodo(periodo: str) -> tuple[int, int]:
    """
    Devuelve (inicio, fin) a partir de 'YYYY-YYYY'.
    Lanza ValueError si el formato no calza.
    """
    m = PERIODO_RE.match(str(periodo))
    if not m:
        raise ValueError(f"Formato de Periodo inválido: {periodo!r}")
    ini, fin = int(m.group(1)), int(m.group(2))
    if fin < ini:
        raise ValueError(f"Año final < inicial en Periodo: {periodo!r}")
    return ini, fin


def safe_concat(frames: Iterable[pd.DataFrame], **kwargs) -> pd.DataFrame:
    """
    Concatena ignorando None / DFs vacíos.
    """
    lst = [df for df in frames if df is not None and not df.empty]
    if not lst:
        return pd.DataFrame()
    return pd.concat(lst, ignore_index=True, **kwargs)

def build_detalle_periodo(nombre_periodo: str) -> pd.DataFrame:
    """
    Construye el detalle de votos para un periodo 'YYYY-YYYY':
      - junta votos por año
      - trae detalle por Id
      - explota Votos.Voto y normaliza
      - devuelve DataFrame final listo para guardar
    """
    inicio, fin = parse_periodo(nombre_periodo)

    # Incluimos ambos extremos del periodo (YYYY-YYYY)
    years = range(inicio, fin + 1)

    # 1) Votos por año
    df_vot_year = safe_concat([get_vot_year(y) for y in years])
    if df_vot_year.empty:
        return pd.DataFrame()

    if "Id" not in df_vot_year.columns:
        raise KeyError("La columna 'Id' no existe en df_vot_year.")

    # 2) Detalle por cada Id
    detalles = [get_detalle(idx) for idx in df_vot_year["Id"]]
    df_detalle = safe_concat(detalles)
    if df_detalle.empty:
        return pd.DataFrame()

    # 3) Explode y normalización
    col_votos = "Votos.Voto"
    if col_votos not in df_detalle.columns:
        # Nada que explotar; devolvemos el detalle tal cual
        return df_detalle.reset_index(drop=True)

    df_explode = df_detalle.explode(col_votos, ignore_index=True)

    # Algunas filas podrían no tener Votos.Voto (NaN) → normalizamos solo las no nulas
    votos_series = df_explode[col_votos]
    has_voto = votos_series.notna()

    if has_voto.any():
        # normalizamos únicamente las filas con dato
        df_votos_norm = json_normalize(votos_series[has_voto])
        # creamos un df vacío con el mismo índice para alinear
        df_votos_full = pd.DataFrame(index=df_explode.index)
        df_votos_full.loc[has_voto, df_votos_norm.columns] = df_votos_norm.values
        # concatenamos columnas normalizadas al explode original
        df_final = pd.concat([df_explode.drop(columns=[col_votos]), df_votos_full], axis=1)
    else:
        # No había votos anidados
        df_final = df_explode.drop(columns=[col_votos])

    return df_final.reset_index(drop=True)

# === Loop principal por periodos ===
# Usamos itertuples para evitar el bug de 'rows/row' y ganar velocidad
for row in df_periodos.itertuples(index=False):
    nombre_periodo = getattr(row, "Periodo", None)
    if pd.isna(nombre_periodo):
        print("⏭️  Saltando fila sin 'Periodo'.")
        continue

    try:
        df_out = build_detalle_periodo(str(nombre_periodo))
    except Exception as e:
        print(f"❌ Error procesando periodo {nombre_periodo!r}: {e}")
        continue

    carpeta = DATA_DIR / sanitize(str(nombre_periodo))  # p.ej., data/1990-1994
    carpeta.mkdir(parents=True, exist_ok=True)

    out_path = carpeta / "detalle.csv"
    if df_out.empty:
        print(f"⚠️  Periodo {nombre_periodo}: no se generaron filas. Archivo NO creado.")
        continue

    # Guardado prolijo
    df_out.to_csv(out_path, index=False, encoding="utf-8", lineterminator="\n")
    print(f"✓ Guardado {out_path}  (filas: {len(df_out):,})")


⚠️  Periodo 1990-1994: no se generaron filas. Archivo NO creado.
⚠️  Periodo 1994-1998: no se generaron filas. Archivo NO creado.


C:\Users\angel\AppData\Local\Temp\ipykernel_15288\408016535.py:82: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Enrique', 'Sergio', 'Rodrigo', 'Pedro Pablo', 'Ramón', 'Germán', 'Pablo', 'Sergio', 'Rosa', 'Carlos', 'José Antonio', 'Cristián', 'Darío', 'Eduardo', 'Alejandra', 'Gastón', 'Esteban', 'Samuel', 'Eugenio', 'Mario', 'Jorge', 'Juan', 'Eliana', 'Alberto', 'Guillermo', 'María Angélica', 'Marcela', 'Julio', 'Francisco', 'Fidel', 'Marcelo', 'Rodrigo', 'Gonzalo', 'Enrique', 'Antonio', 'Zarko', 'María Eugenia', 'Carlos', 'Pedro', 'Sergio', 'José Miguel', 'Iván', 'Cristián', 'Darío', 'Ramón', 'Víctor', 'Pablo', 'Edgardo', 'María Antonieta', 'Felipe', 'Edmundo', 'Leopoldo', 'Exequiel', 'Laura', 'Boris', 'Carolina', 'Eugenio', 'Gonzalo', 'Ignacio', 'Alfonso', 'Ximena', 'Carlos', 'Patricio', 'Isabel', 'Pedro', 'Claudio', 'Rodrigo', 'Pedro Pablo', 'Gabriel', 'Pablo', 'Rosa', 'Carlos', 'José Antonio', 'Iván', 'Eduard

✓ Guardado C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative Voting Behavior Prediction\data\1998-2002\detalle.csv  (filas: 35,934)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



✓ Guardado C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative Voting Behavior Prediction\data\2002-2006\detalle.csv  (filas: 256,640)


...

✓ Guardado C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative Voting Behavior Prediction\data\2014-2018\detalle.csv  (filas: 553,908)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



✓ Guardado C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative Voting Behavior Prediction\data\2018-2022\detalle.csv  (filas: 854,659)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



✓ Guardado C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative Voting Behavior Prediction\data\2022-2026\detalle.csv  (filas: 770,471)


### Obtener biografías de parlamentarios 

In [9]:
df = pd.read_csv(DATA_DIR / "2022-2026/diputados.csv")

In [10]:
df.head()

,periodo_id,Periodo,diputado_id,nombre_completo,fecha_nacimiento,sexo,edad,distrito_numero,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino
0,10,2022-2026,1165,Agustín Romero Leiva,1975-01-03,Masculino,47,NaN,PREP,Partido Republicano,0,2022-03-11,2026-03-10 23:59:59
1,10,2022-2026,1179,Alberto Undurraga Vicuña,1969-06-17,Masculino,52,NaN,DC,Partido Demócrata Cristiano,0,2022-03-11,2026-03-10 23:59:59
2,10,2022-2026,1160,Alejandra Placencia Cabello,1978-02-14,Femenino,44,NaN,PC,Partido Comunista,0,2022-03-11,2026-03-10 23:59:59
3,10,2022-2026,1013,Alejandro Bernales Maldonado,1979-02-02,Masculino,43,NaN,LIBERAL,Partido Liberal de Chile,1,2022-03-11,2026-03-10 23:59:59
4,10,2022-2026,1076,Alexis Sepúlveda Soto,1969-12-29,Masculino,52,NaN,PR,Partido Radical de Chile,1,2022-03-11,2026-03-10 23:59:59


In [66]:
BASE = "https://www.bcn.cl/historiapolitica/resenas_parlamentarias/index.html"
COMMON_PARAMS = {
    "categ": "en_ejercicio",
    "filtros": "3",
    "K": "1",
    # "pagina": 1  # la seteamos en el loop
}

HEADERS = {"User-Agent": "Mozilla/5.0"}

def fetch_page(pagina: int):
    params = {**COMMON_PARAMS, "pagina": str(pagina)}
    r = requests.get(BASE, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text, r.url  # html y url final (útil para urljoin)

def parse_cards(html: str, base_url: str):
    soup = BeautifulSoup(html, "html.parser")
    items = []
    # cada “tarjeta” de parlamentario
    for card in soup.select('#contenedorResultados .thumb-item'):
        # link a la wiki (puede venir relativo)
        a = card.find('a', href=True)
        if not a:
            continue
        url = urljoin(base_url, a['href'])

        # nombre (suele venir en h4 > a, pero dejamos fallback)
        h4a = card.select_one('h4 a')
        nombre = h4a.get_text(strip=True) if h4a else a.get_text(strip=True)

        items.append({"nombre_en_lista": nombre, "url_wiki": url})
    return items

def scrape_bcn_list(pagina_inicial=1, pagina_final=10, sleep=1.0):
    rows = []
    for p in range(pagina_inicial, pagina_final + 1):
        html, resolved_url = fetch_page(p)
        cards = parse_cards(html, resolved_url)
        if not cards:  # se acabaron los resultados
            break
        for it in cards:
            it["pagina"] = p
            rows.append(it)
        time.sleep(sleep)  # ser amable con el servidor
    return pd.DataFrame(rows)

def fetch_bcn(nombre: str, sleep=1.0) -> dict:
    """
    Intenta con URL directa; si 404, usa búsqueda.
    Devuelve dict con campos parseados + url usada.
    """
    url = build_bcn_url(nombre)
    headers = {"User-Agent":"Mozilla/5.0"}
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if r.status_code == 404:
        print("❌ No existe:", nombre)
        return None
    soup = BeautifulSoup(r.text, "html.parser")
    span_distrito = soup.find("span", {"property": "bcnbio:representingPlaceNamed"})
    if span_distrito:
        txt_distrito = span_distrito.text.strip()
        distrito = int(re.search(r"\d+", txt_distrito).group())
    else:
        print("❌ No existe span_distrito:", nombre)
    span_profesion = soup.find("span", {"property": "bcnbio:profession"})
    if span_profesion:
        profesion = span_profesion.text.strip()
    else:
        print("❌ No existe span_profesion:", nombre)
    
    return url

# === Ejecutar ===
df_parlamentarios = scrape_bcn_list(pagina_inicial=1, pagina_final=6, sleep=0.6)
print(len(df_parlamentarios))
df_parlamentarios.head()

155


,nombre_en_lista,url_wiki,pagina
0,María Candelaria Acevedo Sáez,https://www.bcn.cl/historiapolitica/resenas_pa...,1
1,Eric Aedo Jeldres,https://www.bcn.cl/historiapolitica/resenas_pa...,1
2,Yovana Ahumada Palma,https://www.bcn.cl/historiapolitica/resenas_pa...,1
3,Jorge Iván Alessandri Vergara,https://www.bcn.cl/historiapolitica/resenas_pa...,1
4,René Alinco Bustos,https://www.bcn.cl/historiapolitica/resenas_pa...,1


In [92]:
HEADERS = {"User-Agent": "Mozilla/5.0"}

def norm(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip(" \t\n\r")

def fetch_html(url: str, sleep=0.25):
    r = requests.get(url, headers=HEADERS, timeout=30)
    time.sleep(sleep)
    return r.status_code, r.text

def get_section_paragraphs(soup: BeautifulSoup, title_patterns):
    """
    Busca div.box_contenidos con <h4> cuyo texto calce con alguno de title_patterns (regex o strings),
    y devuelve una lista de párrafos (texto limpio) dentro de ese box.
    """
    if not isinstance(title_patterns, (list, tuple)):
        title_patterns = [title_patterns]

    paras = []
    for box in soup.select("div.box_contenidos"):
        h4 = box.find("h4")
        if not h4:
            continue
        title = norm(h4.get_text(" ", strip=True))
        if any(re.search(p, title, flags=re.I) for p in title_patterns):
            # en tu HTML: h4 + div con <p>… Tomamos todos los <p> dentro del box.
            for p in box.find_all("p"):
                t = norm(p.get_text(" ", strip=True))
                if t:
                    paras.append(t)
    return paras

def extract_paragraphs_from_url(url: str, sleep=0.25):
    out = {
        "status": None,
        "familia_juventud_parrafos": [],
        "estudios_vida_laboral_parrafos": [],
    }
    status, html = fetch_html(url, sleep=sleep)
    out["status"] = status
    if status != 200:
        return out
    soup = BeautifulSoup(html, "html.parser")

    # Captura exacta de tus secciones
    fam = get_section_paragraphs(soup, [r"Familia\s+y\s+Juventud", r"Familia", r"Juventud"])
    est = get_section_paragraphs(soup, [r"Estudios\s+y\s+vida\s+laboral", r"Estudios", r"Vida\s+laboral"])

    out["familia_juventud_parrafos"] = fam
    out["estudios_vida_laboral_parrafos"] = est
    return out

In [93]:
res = df_parlamentarios['url_wiki'].apply(extract_paragraphs_from_url)
df_parrafos = pd.DataFrame(list(res))
df_check = pd.concat([df_parlamentarios.reset_index(drop=True), df_parrafos], axis=1)

# Muestra rápida (ver párrafos concatenados)
preview = df_check[['nombre_en_lista','familia_juventud_parrafos','estudios_vida_laboral_parrafos']].copy()
preview['familia_juventud_preview'] = preview['familia_juventud_parrafos'].map(lambda xs: " / ".join(xs[:3]) if xs else "")
preview['estudios_vida_preview'] = preview['estudios_vida_laboral_parrafos'].map(lambda xs: " / ".join(xs[:3]) if xs else "")

# Ver 10 filas para inspección
print(preview[['nombre_en_lista','familia_juventud_preview','estudios_vida_preview']].head(10).to_string(index=False))

               nombre_en_lista                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     familia_juventud_preview                                                                                                                                                                                             

In [112]:
import re

def norm(s: str) -> str:
    s = (s or "").replace("\xa0"," ").replace("\u200b"," ")
    s = re.sub(r"\[\d+\]", "", s)          # quita [1], [2], ...
    s = re.sub(r"\s+", " ", s)
    return s.strip(" \t\n\r,;:.-–—")

# Instituciones (muy permisivo)
PAT_INSTIT = re.compile(
    r"\b(?P<tipo>Colegio|Escuela|Liceo|Instituto|Universidad)\b(?P<resto>[^.;\n]+)",
    re.I | re.U
)

# Áreas/estudios (evita “Estudió en …”)
PAT_FIELD = re.compile(
    r"("
    r"licenciad[oa]\s+en\s+[^.;\n]+|"
    r"se\s+licenci[óo]\s+en\s+[^.;\n]+|"
    r"licenci[óo]\s+en\s+[^.;\n]+|"
    r"carrera\s+de\s+[^.;\n]+|"
    r"titulad[oa]\s+en\s+[^.;\n]+|"
    r"se\s+titul[óo]\s+de\s+[^.;\n]+|"
    r"t[íi]tulo\s+de\s+[^.;\n]+|"
    r"mag[íi]ster\s+en\s+[^.;\n]+|"
    r"maestr[íi]a\s+en\s+[^.;\n]+|"
    r"doctorado\s+en\s+[^.;\n]+|"
    r"post[ií]tulo\s+en\s+[^.;\n]+|"
    r"diplomado\s+en\s+[^.;\n]+|"
    r"estudi(?:[óo]|aba|a)\s+(?!en\b)[A-Za-zÁÉÍÓÚÑáéíóúñ]+(?:\s+de\s+[A-Za-zÁÉÍÓÚÑáéíóúñ]+)*"
    r")",
    re.I | re.U
)

PAT_WORK = re.compile(
    r"(trabaj(?:[óo]|a)\s+en\s+[^.;\n]+|se\s+desempeñ(?:[óo]|a)\s+como\s+[^.;\n]+|"
    r"ocup(?:[óo]|a)\s+el\s+cargo\s+de\s+[^.;\n]+|empresa\s+de\s+[^.;\n]+|"
    r"director[io]?\s+de\s+[^.;\n]+)",
    re.I | re.U
)

def to_sentences(text):
    text = norm(text)
    return [t for t in re.split(r"(?<=[\.\?\!;])\s+(?=[A-ZÁÉÍÓÚÑ\"(])", text) if t.strip()]

def extract_from_section_paragraphs(paragraphs):
    escolaridad, universidades, fields, trabajos = [], [], [], []
    for par in paragraphs:
        for s in to_sentences(par):
            # instituciones
            for m in PAT_INSTIT.finditer(s):
                inst = norm(m.group("tipo") + m.group("resto"))
                inst = re.sub(r"\s*\((?:\d{4}[-–]\d{4}|\d{4})\)\s*$", "", inst).strip()
                if inst.lower().startswith("universidad"):
                    if inst not in universidades: universidades.append(inst)
                else:
                    if inst not in escolaridad: escolaridad.append(inst)
            # campos
            for m in PAT_FIELD.finditer(s):
                val = norm(m.group(1))
                if val not in fields: fields.append(val)
            # trabajo
            for m in PAT_WORK.finditer(s):
                val = norm(m.group(1))
                if val not in trabajos: trabajos.append(val)
    return escolaridad, universidades, fields, trabajos

# --- aplicar a tu DF (usar AMBAS secciones) ---
def parse_from_row(row):
    # unimos ambas secciones porque BCN a veces mezcla info
    pars = []
    if isinstance(row.get('estudios_vida_laboral_parrafos'), list):
        pars += row['estudios_vida_laboral_parrafos']
    if isinstance(row.get('familia_juventud_parrafos'), list):
        pars += row['familia_juventud_parrafos']
    esc, uni, fields, work = extract_from_section_paragraphs(pars)
    return pd.Series({
        "escolaridad": esc,
        "universidades": uni,
        "estudios_mencionados": fields,
        "trabajo": work
    })


In [116]:
# 1) Aplicar el parser fila a fila usando ambas secciones de párrafos
df_info = df_check.apply(parse_from_row, axis=1)

# 2) Unir resultados al DF original
df_final = pd.concat([df_check.reset_index(drop=True), df_info], axis=1)

# 3) Vista rápida para verificar
cols_ver = ['nombre_en_lista','escolaridad','universidades','estudios_mencionados','trabajo']
df_final[cols_ver]

,nombre_en_lista,escolaridad,universidades,estudios_mencionados,trabajo
0,María Candelaria Acevedo Sáez,"[Escuela Clorinda Avello, de Santa Juana (Prov...",[],[estudiaba contabilidad],[]
1,Eric Aedo Jeldres,"[Liceo de Hombres A-59, actual Liceo Bicentena...",[Universidad Católica de la Santísima Concepción],[se tituló de Administrador de Programas Socia...,[director de Apoyo a los Estudiantes (DAE) en ...
2,Yovana Ahumada Palma,"[Escuela D-91, luego en el Colegio Antofagasta...",[],[],[se desempeñó como agente de venta de seguros ...
3,Jorge Iván Alessandri Vergara,[Colegio Internacional Nido de Águilas de Sant...,"[Universidad de Los Andes (Chile), donde se Li...",[carrera de Derecho en la Universidad de Los A...,[empresa de seguridad que da trabajo a un impo...
4,René Alinco Bustos,[Escuela D-21 y en el Liceo Juan Gómez Millas ...,[],[],[trabajó en instalaciones petroleras y mineras...
...,...,...,...,...,...
150,Sebastián Videla Castillo,"[Colegio Advestista de Antofagasta, de donde e...",[Universidad Católica de Valparaíso],"[estudió actuación, Diplomado en Gestión Cultu...",[trabajó en Chilevisión]
151,Gastón Von Mühlenbrock Zamora,[Colegio Luis Campino de Santiago],"[Universidad de Chile, donde egresó de la carr...",[carrera de Ingeniería Comercial y obtuvo el g...,[trabajó en el Ministerio del Interior]
152,Flor Weisse Novoa,"[Liceo N° 1 de Niñas de Concepción, establecim...",[Universidad de Concepción],[estudió ingeniería],[]
153,Gonzalo Rodolfo Winter Etcheberry,[Colegio Verbo Divino de Santiago del cual egr...,"[Universidad de Chile, Universidad de Chile, g...",[],[]


In [119]:
cols = ['nombre_en_lista','universidades','escolaridad','estudios_mencionados','trabajo']

df_final[cols] = df_final[cols].applymap(
    lambda x: pd.NA if (x is None or (isinstance(x, (list, tuple, set)) and len(x)==0)) else x
)

nulos_por_col = df_final[cols].isna().sum()
print(nulos_por_col)

nombre_en_lista          0
universidades           23
escolaridad             13
estudios_mencionados    33
trabajo                 91
dtype: int64


C:\Users\angel\AppData\Local\Temp\ipykernel_2056\1770346680.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final[cols] = df_final[cols].applymap(
